In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
df = pd.read_csv("/kaggle/input/System-Threat-Forecaster/train.csv")
test_data = pd.read_csv("/kaggle/input/System-Threat-Forecaster/test.csv")
sample_submission = pd.read_csv('/kaggle/input/System-Threat-Forecaster/sample_submission.csv')
df.head()

In [ ]:
from sklearn.model_selection import train_test_split

# Assuming `X` is your feature matrix and `y` is your target variable
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV
import numpy as np

# Assuming you have X (features) and y (target)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Define search space
param_dist = {
    'n_estimators': np.arange(100, 500, 50),
    'learning_rate': np.linspace(0.01, 0.3, 5),
    'max_depth': np.arange(3, 10),
    'subsample': np.linspace(0.6, 1.0, 3),
    'colsample_bytree': np.linspace(0.6, 1.0, 3),
}

# Initialize model
xgb_clf = XGBClassifier(use_label_encoder=False, eval_metric='logloss')

# Randomized Search
random_search = RandomizedSearchCV(
    xgb_clf, param_distributions=param_dist, 
    n_iter=10, cv=3, scoring='accuracy', n_jobs=-1, verbose=1
)

# Fit the model
random_search.fit(X_train, y_train)

# Best parameters
print("Best parameters:", random_search.best_params_)

# Use the best model
best_model = random_search.best_estimator_


In [ ]:
df.info

In [ ]:
#df =df.dropna(subset=['target'])
#df.info()

In [ ]:
from sklearn.impute import SimpleImputer

# Identify numerical and categorical columns
num_cols = df.select_dtypes(include=['int64', 'float64']).columns
cat_cols = df.select_dtypes(include=['object']).columns
num_cols = num_cols.drop('target', errors='ignore')

# Impute numerical columns with mean
imputer_num = SimpleImputer(strategy='mean')
df[num_cols] = imputer_num.fit_transform(df[num_cols])
test_data[num_cols] = imputer_num.transform(test_data[num_cols])

# Impute categorical columns with most frequent value
imputer_cat = SimpleImputer(strategy='most_frequent')
df[cat_cols] = imputer_cat.fit_transform(df[cat_cols])
test_data[cat_cols] = imputer_cat.transform(test_data[cat_cols])

print("Missing values handled successfully!")

In [ ]:
df['DateAS'] = pd.to_datetime(df['DateAS']).dt.month
df['DateOS'] = pd.to_datetime(df['DateOS']).dt.month
test_data['DateAS'] = pd.to_datetime(test_data['DateAS']).dt.month
test_data['DateOS'] = pd.to_datetime(test_data['DateOS']).dt.month

In [ ]:
cardinality = df.nunique().sort_values(ascending=False)

low_cardinality = cardinality[cardinality == 1].index.tolist()
df = df.drop(columns=low_cardinality)
test_data = test_data.drop(columns=low_cardinality)
df.shape

In [ ]:
categorical_cols = df.select_dtypes(include=['object']).columns

selected_features = [col for col in categorical_cols if df[col].nunique() <= 10]
high_cardinality_features = [col for col in categorical_cols if df[col].nunique() > 10]
df = pd.get_dummies(df, columns=selected_features, drop_first=True)
test_data= pd.get_dummies(test_data, columns=selected_features, drop_first=True)
df

In [ ]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
for col in high_cardinality_features:
    # Apply Label Encoding
    df[col] = encoder.fit_transform(df[col])
    test_data[col] = encoder.fit_transform(test_data[col]) 

# Display the transformed dataset
df.head()

In [ ]:
num_cols = df.select_dtypes(include=['int64','int32','float64']).columns.drop('target', errors='ignore')
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df[num_cols] = scaler.fit_transform(df[num_cols])
test_data[num_cols] = scaler.fit_transform(test_data[num_cols]) 

In [ ]:
df= df.drop(columns=list(set(df.columns)-(set(test_data.columns)|{'target'})), axis=1)

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder, StandardScaler, SimpleImputer
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score

# Load dataset (Replace 'your_data.csv' with actual file path)
df = pd.read_csv('your_data.csv')

# Separate features (X) and target (y)
X = df.drop(columns=['target_column'])  # Replace 'target_column' with actual target column name
y = df['target_column']

# Identify categorical and numeric columns
cat_cols = X.select_dtypes(include=['object']).columns
num_cols = X.select_dtypes(include=['int64', 'float64']).columns

# Impute missing values
cat_imputer = SimpleImputer(strategy='most_frequent')
num_imputer = SimpleImputer(strategy='mean')
X[cat_cols] = cat_imputer.fit_transform(X[cat_cols])
X[num_cols] = num_imputer.fit_transform(X[num_cols])

# Label encode categorical features
label_encoders = {}
for col in cat_cols:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col])
    label_encoders[col] = le

# Scale numeric features
scaler = StandardScaler()
X[num_cols] = scaler.fit_transform(X[num_cols])

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# --- Model 1: Decision Tree ---
dt = DecisionTreeClassifier(random_state=42)
param_grid_dt = {
    'max_depth': [20, 30],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}

grid_dt = GridSearchCV(dt, param_grid_dt, cv=3, scoring='accuracy')
grid_dt.fit(X_train, y_train)

# Best parameters for Decision Tree
best_max_depth = grid_dt.best_params_['max_depth']
best_min_samples_split = grid_dt.best_params_['min_samples_split']
best_min_samples_leaf = grid_dt.best_params_['min_samples_leaf']

dt_best = grid_dt.best_estimator_
y_pred_dt = dt_best.predict(X_test)
dt_accuracy = round(accuracy_score(y_test, y_pred_dt), 2)

# --- Model 2: AdaBoost ---
adaboost = AdaBoostClassifier(random_state=42)
param_grid_ab = {
    'n_estimators': [10, 20, 30],
    'learning_rate': [5, 10],
    'algorithm': ['SAMME']
}

grid_ab = GridSearchCV(adaboost, param_grid_ab, cv=3, scoring='accuracy')
grid_ab.fit(X_train, y_train)

# Best parameters for AdaBoost
best_n_estimators = grid_ab.best_params_['n_estimators']
best_learning_rate = grid_ab.best_params_['learning_rate']

ab_best = grid_ab.best_estimator_
y_pred_ab = ab_best.predict(X_test)
ab_accuracy = round(accuracy_score(y_test, y_pred_ab), 2)

# Output results
print(f"Best max_depth for Decision Tree: {best_max_depth}")
print(f"Best min_samples_split for Decision Tree: {best_min_samples_split}")
print(f"Best min_samples_leaf for Decision Tree: {best_min_samples_leaf}")
print(f"Decision Tree Accuracy: {dt_accuracy}")

print(f"Best n_estimators for AdaBoost: {best_n_estimators}")
print(f"Best learning_rate for AdaBoost: {best_learning_rate}")
print(f"AdaBoost Accuracy: {ab_accuracy}")


In [ ]:
from sklearn.preprocessing import LabelEncoder

# Automatically identify categorical columns
categorical_cols = df.select_dtypes(include=['object']).columns
test_categorical_cols = test_data.select_dtypes(include=['object']).columns

# Apply Label Encoding
encoder = LabelEncoder()
for col in categorical_cols:
    df[col] = encoder.fit_transform(df[col])
for col in test_categorical_cols:
    test_data[col] = encoder.fit_transform(test_data[col])    

In [ ]:
from sklearn.model_selection import train_test_split
X = df.drop(columns=['target'])   # Features
y = df['target']     # Target column
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

# Initialize the XGBoost classifier
xgb = XGBClassifier(n_estimators=500, random_state=42)

# Train the model
xgb.fit(X_train, y_train)
y_pred_xgb = xgb.predict(X_test)
accuracy = accuracy_score(y_test, y_pred_xgb)
print(f"Accuracy: {accuracy:.4f}")

In [ ]:
test_data = test_data[X.columns]

In [ ]:
#import pandas as pd
#df = pd.read_csv("/kaggle/input/System-Threat-Forecaster/train.csv")
#df_copy = df.copy()

In [ ]:
#constant_columns = df.nunique()[df.nunique() == 1]
#print(constant_columns)

In [ ]:
#import pandas as pd
#df = pd.read_csv("/kaggle/input/System-Threat-Forecaster/train.csv")
#test_data = pd.read_csv("/kaggle/input/System-Threat-Forecaster/train.csv")
#sample_submission = pd.read_csv('/kaggle/input/System-Threat-Forecaster/sample_submission.csv')
#df.head()


In [ ]:
#df =df.dropna(subset=['target'])
#df.info()



In [ ]:
test_predictions = xgb.predict(test_data)  # Generate predictions

# Create submission DataFrame (Ensure sample_submission exists)
submission = sample_submission.copy()
submission['target'] = test_predictions  

# If classification-like labels are needed (Assuming binary case)
# Ensure model output is properly rounded or thresholded
submission['target'] = submission['target'].apply(lambda x: '1' if x > 0.5 else '0')

# Save to CSV
submission.to_csv('submission.csv', index=False)

# Output file ready for submission
print("Submission file created.")

In [ ]:
df =pd.read_csv("/kaggle/input/System-Threat-Forecaster/train.csv")


In [ ]:
import pandas as pd
X_test =pd.read_csv("/kaggle/input/System-Threat-Forecaster/test.csv")




df = pd.read_csv("/kaggle/input/System-Threat-Forecaster/train.csv")


df_clean = df.dropna()





X =df.drop("target", axis =1)
y =df["target"]

#from sklearn.dummy import DummyClassifier
#model =DummyClassifier().fit(X,y)
#y_pred =model.predict(X_test)

In [ ]:
#submission =pd.DataFrame({"id": range(0,X_test.shape[0]),

#"target": y_pred})

#submission.to_csv('submission.csv', index =False)
